In [11]:
#Basic import
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
#Modelling
from sklearn.metrics import mean_squared_error,r2_score,mean_absolute_error
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction import DictVectorizer
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression,Ridge,Lasso
from sklearn.model_selection import RandomizedSearchCV
from catboost import CatBoostRegressor
from sklearn.metrics import roc_auc_score

from xgboost import XGBRegressor
import warnings
warnings.filterwarnings('ignore')

In [12]:
df=pd.read_csv("cleaned_data.csv")
df

,Unnamed: 0,carat,cut,color,clarity,depth,table,x,y,z,price
0,0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,2401.0
1,1,0.70,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05,2401.0
2,2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772.0
3,3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71,666.0
4,4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77,2401.0
...,...,...,...,...,...,...,...,...,...,...,...
193568,193568,0.31,Ideal,D,VVS2,61.1,56.0,4.35,4.39,2.67,1130.0
193569,193569,0.70,Premium,G,VVS2,60.3,58.0,5.75,5.77,3.47,2874.0
193570,193570,0.73,Very Good,F,SI1,63.1,57.0,5.72,5.75,3.62,3036.0
193571,193571,0.34,Very Good,D,SI1,62.9,55.0,4.45,4.49,2.81,681.0


In [13]:
df.isna().sum()

Unnamed: 0    0
carat         0
cut           0
color         0
clarity       0
depth         0
table         0
x             0
y             0
z             0
price         0
dtype: int64

In [14]:
## Prepare X and y values
X=df.drop('price',axis=1)
y=df['price']

In [15]:
X

,Unnamed: 0,carat,cut,color,clarity,depth,table,x,y,z
0,0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55
1,1,0.70,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05
2,2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50
3,3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71
4,4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77
...,...,...,...,...,...,...,...,...,...,...
193568,193568,0.31,Ideal,D,VVS2,61.1,56.0,4.35,4.39,2.67
193569,193569,0.70,Premium,G,VVS2,60.3,58.0,5.75,5.77,3.47
193570,193570,0.73,Very Good,F,SI1,63.1,57.0,5.72,5.75,3.62
193571,193571,0.34,Very Good,D,SI1,62.9,55.0,4.45,4.49,2.81


In [16]:
y

0         2401.0
1         2401.0
2         2772.0
3          666.0
4         2401.0
           ...  
193568    1130.0
193569    2874.0
193570    3036.0
193571     681.0
193572    2258.0
Name: price, Length: 193573, dtype: float64

In [17]:
categorical_columns=X.select_dtypes(include='object').columns
categorical_columns

Index(['cut', 'color', 'clarity'], dtype='object')

In [18]:
#numerical_columns=X.select_dtypes(exclude='object')
numerical_columns=['carat','depth','x','y','z']

In [19]:
X.select_dtypes(include='object')

,cut,color,clarity
0,Premium,F,VS2
1,Very Good,J,SI2
2,Ideal,G,VS1
3,Ideal,G,VS1
4,Premium,G,VS2
...,...,...,...
193568,Ideal,D,VVS2
193569,Premium,G,VVS2
193570,Very Good,F,SI1
193571,Very Good,D,SI1


In [20]:
df.color.nunique(),df.color.value_counts()
df.clarity.nunique(),df.clarity.value_counts()


(8,
 SI1     53272
 VS2     48027
 VS1     30669
 SI2     30484
 VVS2    15762
 VVS1    10628
 IF       4219
 I1        512
 Name: clarity, dtype: int64)

In [21]:
# Define the custom ranking for each ordinal variable
cut_categories = ['Fair', 'Good', 'Very Good','Premium','Ideal']
color_categories = ['D', 'E', 'F', 'G', 'H', 'I', 'J']
clarity_categories = ['I1','SI2','SI1','VS2','VS1','VVS2','VVS1','IF']

In [22]:
from sklearn.impute import SimpleImputer ## HAndling Missing Values
from sklearn.preprocessing import StandardScaler # HAndling Feature Scaling
from sklearn.preprocessing import OrdinalEncoder # Ordinal Encoding
## pipelines
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [23]:
num_pipeline=Pipeline(
    steps=[
        ('imputer',SimpleImputer()),
        ('scaler',StandardScaler())
    ]
)



In [24]:
cat_pipeline=Pipeline(
    steps=[
        ('imputer',SimpleImputer(strategy='most_frequent')),
        ('ordinalencoder',OrdinalEncoder(categories=[cut_categories,color_categories,clarity_categories])),
        ('scaler',StandardScaler())
    ]
)



In [25]:
preprocessor=ColumnTransformer(
    [
        ('num_pipeline',num_pipeline,numerical_columns),
        ('cat_pipeline',cat_pipeline,categorical_columns)
    ]
)



In [26]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.30,random_state=30)

In [27]:
X_train=preprocessor.fit_transform(X_train)
X_train

array([[-1.0403135 , -1.16911942, -1.04335962, ...,  0.87407553,
         1.52872212,  1.35273128],
       [ 0.42798685,  2.29489642,  0.31880529, ..., -2.14455824,
        -0.93507064, -0.64678628],
       [ 0.74262264,  0.98095938,  0.57139216, ..., -0.13213573,
         0.29682574,  0.68622543],
       ...,
       [ 0.69018334,  1.93654995,  0.5172664 , ..., -2.14455824,
         0.29682574, -0.64678628],
       [ 0.95237983, -2.36360764,  0.86908382, ...,  0.87407553,
         0.29682574,  0.68622543],
       [ 0.4542065 ,  0.98095938,  0.381952  , ...,  0.87407553,
         0.29682574, -0.64678628]])

In [28]:
X_test=preprocessor.transform(X_test)
X_test

array([[-0.54214016, -1.28856824, -0.42993437, ..., -0.13213573,
        -0.93507064,  0.01971957],
       [-0.07018648,  1.21985702, -0.0420331 , ..., -1.13834698,
         0.91277393, -0.64678628],
       [-1.14519209,  0.26426645, -1.30496745, ...,  0.87407553,
         0.91277393,  2.68574299],
       ...,
       [-0.09640613,  0.0253688 ,  0.10230225, ..., -0.13213573,
         2.14467031,  0.01971957],
       [-0.85677595, -0.33297766, -0.85391947, ...,  0.87407553,
         0.29682574, -0.64678628],
       [-0.09640613, -1.04967059,  2.14103913, ..., -0.13213573,
         2.14467031,  0.01971957]])

In [29]:
"""linear regression
ridge regression
lasso regression
elastic net"""

'linear regression\nridge regression\nlasso regression\nelastic net'

In [30]:
from sklearn.linear_model import LinearRegression,Lasso,Ridge,ElasticNet
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error

In [31]:
def evaluate_model(true,predicted):
    mae=mean_absolute_error(true,predicted)
    mse=mean_squared_error(true,predicted)
    rmse=np.sqrt(mean_squared_error(true,predicted))
    r2_square=r2_score(true,predicted)
    return mae,rmse,r2_square

In [32]:
## Train multiple models
models={
    'LinearRegression':LinearRegression(),
    'lasso':Lasso(),
    'Ridge':Ridge(),
    'Elasticnet':ElasticNet()
}

In [33]:
list(models)

['LinearRegression', 'lasso', 'Ridge', 'Elasticnet']

In [34]:
for i in range (len(list(models))):
    model=list(models.values())[i]
    print(model)

LinearRegression()
Lasso()
Ridge()
ElasticNet()


In [35]:
models.keys()

dict_keys(['LinearRegression', 'lasso', 'Ridge', 'Elasticnet'])

In [36]:
models.values()

dict_values([LinearRegression(), Lasso(), Ridge(), ElasticNet()])

In [37]:
X_train

array([[-1.0403135 , -1.16911942, -1.04335962, ...,  0.87407553,
         1.52872212,  1.35273128],
       [ 0.42798685,  2.29489642,  0.31880529, ..., -2.14455824,
        -0.93507064, -0.64678628],
       [ 0.74262264,  0.98095938,  0.57139216, ..., -0.13213573,
         0.29682574,  0.68622543],
       ...,
       [ 0.69018334,  1.93654995,  0.5172664 , ..., -2.14455824,
         0.29682574, -0.64678628],
       [ 0.95237983, -2.36360764,  0.86908382, ...,  0.87407553,
         0.29682574,  0.68622543],
       [ 0.4542065 ,  0.98095938,  0.381952  , ...,  0.87407553,
         0.29682574, -0.64678628]])

In [38]:
trained_model_list=[]
model_list=[]
r2_list=[]

for i in range(len(list(models))):
    model=list(models.values())[i]
    model.fit(X_train,y_train)

    #Make Predictions
    y_pred=model.predict(X_test)

    #this is a validation(test) score
    mae, rmse, r2_square=evaluate_model(y_test,y_pred)

    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])

    print('Model Training Performance')
    print("RMSE:",rmse)
    print("MAE:",mae)
    print("R2 score",r2_square*100)

    r2_list.append(r2_square)
    
    print('='*35)
    print('\n')

LinearRegression
Model Training Performance
RMSE: 1443.007536269773
MAE: 755.6412069954127
R2 score 71.83574058207694


lasso
Model Training Performance
RMSE: 1443.0722685891253
MAE: 755.7587960944967
R2 score 71.83321366703113


Ridge
Model Training Performance
RMSE: 1443.0073982751437
MAE: 755.6495751478873
R2 score 71.83574596876585


Elasticnet
Model Training Performance
RMSE: 1590.035145610078
MAE: 976.0402645919187
R2 score 65.80405694039557


